## Packages and Libraries

In [1]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
from torch import tensor
from torch.nn import Linear, Sequential, ReLU, Identity, BatchNorm1d as BN
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree,remove_self_loops
from torch_geometric.data import Data
from torch_geometric.datasets import TUDataset,Planetoid,GNNBenchmarkDataset
from torch_geometric.loader import DataLoader
from torch_scatter import scatter_mean
from torch.autograd.function import InplaceFunction
from torch_geometric.nn import GCNConv,GINConv,global_mean_pool,TopKPooling
import torch_geometric.transforms as T

from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import time
import argparse
import statistics as stat
from tabulate import tabulate
from collections import OrderedDict

# CPU and Enegusage 
import psutil
import itertools
import tracemalloc
import gc




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Functions for Mmeasuring criterias

In [2]:
def get_num_parameters(model: nn.Module, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for param in model.parameters():
        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements

# Function to get CPU usage
def get_cpu_usage():
    return psutil.cpu_percent(interval=1)



# Function to approximate power consumption (Assume some average power usage per CPU percentage point)
def estimate_power_usage(cpu_usage):
    base_power_usage = 10  # Assumed base power usage in watts
    power_per_percent = 0.5  # Assumed additional watts per CPU usage percent
    return base_power_usage + (power_per_percent * cpu_usage)

# The model size based on the number of parameters
def get_model_size_param(model: nn.Module, data_width=32, count_nonzero_only=False) -> int:
    """
    calculate the model size in bits
    :param data_width: #bits per element
    :param count_nonzero_only: only count nonzero weights
    """
    return get_num_parameters(model, count_nonzero_only) * data_width


Byte = 8
KiB = 1024 * Byte
MiB = 1024 * KiB
GiB = 1024 * MiB

In [3]:
import torch

def calculate_model_size_a2q(model, data_width=32, quantization_bits=8):
    """
    Calculate the model size in bits, considering A2Q quantization
    :param model: The PyTorch model
    :param data_width: #bits per element
    :param quantization_bits: #bits used in quantization
    """
    total_params = sum(p.numel() for p in model.parameters())
    param_bits = total_params * (quantization_bits / data_width)
    total_model_size_bits = param_bits
    total_model_size_bytes = total_model_size_bits / 8
  
    return total_model_size_bytes



## GIN Model

In [4]:
import torch
from torch_geometric.nn import GINConv

class GIN(nn.Module):
    def __init__(self, dataset, num_layers, hidden_units, num_deg=1000, init='norm'):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            nn.Sequential(
                nn.Linear(3, hidden_units),
                nn.ReLU(),
                nn.Linear(hidden_units, hidden_units),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_units),
            ),
            train_eps=True,
        )
        self.convs = torch.nn.ModuleList()
        
        for _ in range(num_layers - 1):
            self.convs.append(
                GINConv(
                    nn.Sequential(
                        nn.Linear(hidden_units, hidden_units),
                        nn.ReLU(),
                        nn.Linear(hidden_units, hidden_units),
                        nn.ReLU(),
                        nn.BatchNorm1d(hidden_units),
                    ),
                    train_eps=True,
                )
            )
        self.bn_list = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.bn_list.append(nn.BatchNorm1d(hidden_units))
        
        self.lin1 = nn.Linear(hidden_units, hidden_units)
        self.lin2 = nn.Linear(hidden_units, dataset.num_classes)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        bit_sum=x.new_zeros(1)
        x= self.conv1(x, edge_index)
        x = self.bn_list[0](x)
        i = 1
        for conv in self.convs:
            x= conv(x, edge_index)
            x = self.bn_list[i](x)
            i += 1
        x = global_mean_pool(x, batch)
        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)



# Helpful Function

In [5]:
## New


class NormalizedDegree(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, data):
        deg = degree(data.edge_index[0], dtype=torch.float)
        deg = (deg - self.mean) / self.std
        data.x = deg.view(-1, 1)
        return data


def num_graphs(data):
    if data.batch is not None:
        return data.num_graphs
    else:
        return data.x.size(0)


def train(model, optimizer, loader,a_loss, a_storage=1):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        data = data.to(device)
        #out,bit_sum = model(data)
        out = model(data)
        target = data.y.view(-1)

        # Check if the last batch is smaller
        if out.size(0) != target.size(0):
            target = target[:out.size(0)]  # Truncate target to match output size

        loss = F.cross_entropy(out, target)
        #loss_store = a_loss*F.relu(bit_sum-a_storage)**2
        #loss_store.backward(retain_graph=True)
        loss.backward()
        total_loss += loss.item() * num_graphs(data)
        optimizer.step()
    return total_loss / len(loader.dataset)

##



def eval_acc(model, loader):
    model.eval()
    correct = 0
    for data in loader:        
        data = data.to(device)
        with torch.no_grad():
            #pred = model(data)[0].max(1)[1] 
            pred = model(data).max(1)[1]

        target = data.y.view(-1)
        # Check if the last batch is smaller
        if pred.size(0) != target.size(0):
            target = target[:pred.size(0)]  # Truncate target to match output size                 
        correct+=pred.eq(target).sum().item()
    return correct / len(loader.dataset)


def eval_loss(model, loader):
    model.eval()

    loss = 0
    for data in loader:     
        data = data.to(device)
        with torch.no_grad():
           #out = model(data)[0]
            out = model(data)

        target = data.y.view(-1)

        # Check if the last batch is smaller
        if out.size(0) != target.size(0):
            target = target[:out.size(0)]  # Truncate target to match output size                
        loss += F.cross_entropy(out, target, reduction="sum").item()
     
    return loss / len(loader.dataset)


# Real k_fold
def k_fold(dataset, folds):
    skf = StratifiedKFold(folds, shuffle=True, random_state=12345)

    test_indices, train_indices = [], []
    for _, idx in skf.split(torch.zeros(len(dataset)), dataset.data.y):
        test_indices.append(torch.from_numpy(idx))

    val_indices = [test_indices[i - 1] for i in range(folds)]

    for i in range(folds):
        train_mask = torch.ones(len(dataset), dtype=torch.bool)
        train_mask[test_indices[i]] = 0
        train_mask[val_indices[i]] = 0
        train_indices.append(train_mask.nonzero().view(-1))

    return train_indices, test_indices, val_indices


def load_checkpoint(model, checkpoint):
    if checkpoint != 'No':
        print("loading checkpoint...")
        model_dict = model.state_dict()
        modelCheckpoint = torch.load(checkpoint)
        pretrained_dict = modelCheckpoint['state_dict']
        new_dict = {k: v for k, v in pretrained_dict.items() if ((k in model_dict.keys()))}
        model_dict.update(new_dict)
        print('Total : {}, update: {}'.format(len(pretrained_dict), len(new_dict)))
        model.load_state_dict(model_dict)
        print("loaded finished!")
    return model

## Setting Argument

In [6]:
import sys
import argparse

# Clearing the arguments
sys.argv = ['']


parser = argparse.ArgumentParser()
parser.add_argument('--model',type=str,default='GIN')
parser.add_argument('--gpu_id',type=int,default=0)
parser.add_argument('--dataset_name',type=str,default='PROTEINS')
parser.add_argument('--num_deg',type=int,default=1000)
parser.add_argument('--num_layers', type=int, default=5)
parser.add_argument('--hidden_units',type=int,default=64)
parser.add_argument('--batch-size',type=int,default=64)
parser.add_argument('--bit',type=int,default=4)
parser.add_argument('--max_epoch',type=int,default=100)
parser.add_argument('--max_cycle',type=int,default=2000)
parser.add_argument('--folds',type=int,default=10)
parser.add_argument('--weight_decay',type=float,default=0)
parser.add_argument('--lr',type=float,default=0.01)
parser.add_argument('--a_loss',type=float,default=0.001)
parser.add_argument('--lr_step_size',type=int, default=50)
parser.add_argument('--lr_decay_factor',type=float,default=0.5)
parser.add_argument('--lr_schedule_patience',type=int,default=10)
parser.add_argument('--is_naive',type=bool,default=False)
###############################################################
parser.add_argument('--resume',type=bool,default=True)
parser.add_argument('--store_ckpt',type=bool,default=True)
parser.add_argument('--uniform',type=bool,default=True)
parser.add_argument('--use_norm_quant',type=bool,default=True)
###############################################################
# The target memory size of nodes features
parser.add_argument('--a_storage',type=float,default=1)
# Path to results
parser.add_argument('--result_folder',type=str,default='result')
# Path to checkpoint
parser.add_argument('--check_folder',type=str,default='checkpoint')
# Path to dataset
parser.add_argument('--pathdataset',type=str,default='/')

args = parser.parse_args()
print(args)

Namespace(model='GIN', gpu_id=0, dataset_name='PROTEINS', num_deg=1000, num_layers=5, hidden_units=64, batch_size=64, bit=4, max_epoch=100, max_cycle=2000, folds=10, weight_decay=0, lr=0.01, a_loss=0.001, lr_step_size=50, lr_decay_factor=0.5, lr_schedule_patience=10, is_naive=False, resume=True, store_ckpt=True, uniform=True, use_norm_quant=True, a_storage=1, result_folder='result', check_folder='checkpoint', pathdataset='/')


In [7]:
###############################################################
model = args.model
args.dataset_name='PROTEINS'
dataset_name = args.dataset_name
num_layers = args.num_layers
hidden_units=args.hidden_units
bit=args.bit
max_epoch = args.max_epoch
resume = args.resume


# Path direction
pathresult = args.result_folder+'/'+args.model+'_'+dataset_name
pathcheck = args.check_folder+'/'+args.model+'_'+dataset_name
if not os.path.exists(pathresult):
    os.makedirs(pathresult)
if not os.path.exists(pathcheck):
    os.makedirs(pathcheck)
###############################################################

In [8]:
pathcheck

'checkpoint/GIN_PROTEINS'

## Loading Dataset and Normalization

In [8]:
def get_dataset(dataset_dir, dataset_name):
    dataset = TUDataset(dataset_dir, dataset_name)
    
    if dataset.data.x is None:
        max_degree = 0
        degs = []
        for data in dataset:
            degs += [degree(data.edge_index[0], dtype=torch.long)]
            max_degree = max(max_degree, degs[-1].max().item())

        if max_degree < 1000:
            dataset.transform = T.OneHotDegree(max_degree)
        else:
            deg = torch.cat(degs, dim=0).to(torch.float)
            mean, std = deg.mean().item(), deg.std().item()
            dataset.transform = NormalizedDegree(mean, std)
            
    return dataset   

In [9]:
dataset=get_dataset(args.pathdataset, dataset_name)

C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [10]:

if dataset.data.x is None:
    max_degree = 0
    degs = []
    for data in dataset:
        degs += [degree(data.edge_index[0], dtype=torch.long)]
        max_degree = max(max_degree, degs[-1].max().item())

    if max_degree < 1000:
        dataset.transform = T.OneHotDegree(max_degree)
    else:
        deg = torch.cat(degs, dim=0).to(torch.float)
        mean, std = deg.mean().item(), deg.std().item()
        dataset.transform = NormalizedDegree(mean, std)

In [11]:
def run(bit=32, max_epoch=5):
        args.batch_size=16
        args.max_epoch=5
        args.batch_size=32
        max_acc =0.50
        
     
      
        val_losses, accu, durations = [], [], []
        main_model_accuracy=[]
        main_model_loss=[]
        t_main_model=[]
        Num_parm_main_model=[]
        main_model_size=[]
        main_energy_consumption=[]
        main_cpu_usage=[]
        main_memory_usage=[]
       
 
        #Eva= OrderedDict()
        #Eva=dict()
        # Initialize a dictionary to store all results per iteration
        Eva_iter = {
            "val losses per iter": [],
            "durations per iter": [],
            "main model accuracy per iter": [],
            "time inference of main model per iter": [],
            "number parmameters of main model per iter": [],  # Store the best accuracy for each fold
            "size of main model per iter": [],
            "energy consumption of main model per iter": [],
            "cpu usage of main model per iter": [],
            "total memory usage of main model per iter": [],
            "final_metrics": {}  # Store final metrics (mean, std, etc.)
        }
    
        
     


        for fold, (train_idx, test_idx, val_idx) in enumerate(zip(*k_fold(dataset, args.folds))):
            print_max_acc=0
            train_dataset = dataset[train_idx.tolist()]
            test_dataset = dataset[test_idx.tolist()]
            val_dataset = dataset[val_idx.tolist()]
            train_loader = DataLoader(train_dataset, args.batch_size, num_workers=0,shuffle=False, drop_last=True)
            val_loader = DataLoader(val_dataset, args.batch_size,num_workers=0,shuffle=False,drop_last=True)
            test_loader = DataLoader(test_dataset, args.batch_size,num_workers=0,shuffle=False, drop_last=True)
            k=0


            model=GIN(train_dataset, args.num_layers,hidden_units=args.hidden_units,
                num_deg=args.num_deg).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_step_size, gamma=args.lr_decay_factor)
            
            t_start = time.perf_counter()

            Eva_fold= OrderedDict() #It is a dictionary to arrange output of this fold
        
            for epoch in range(max_epoch):
                #t = tqdm(epoch)
                train_loss=0
                train_loss = train(model,optimizer,train_loader,args.a_loss, args.a_storage)
                start = time.process_time()
                val_loss = eval_loss(model,val_loader)
                val_losses.append(val_loss)
                end = time.process_time()
                acc = eval_acc(model,test_loader)
                

                if epoch % 50 == 0:
                    print(f"Eval Epoch: {epoch} |Val_loss:{val_loss:.03f}| Train_Loss: {train_loss:.3f} | Acc: {acc:.3f}|Fold: {fold}")
                accu.append(acc)
                if(acc>max_acc):
                    max_acc = acc
                    path=pathcheck+'/'+args.model+'_'+dataset_name+'_'+str(bit)+'bit'+'main.pth.tar'
                    torch.save({'state_dict': model.state_dict(), 'best_accu': acc,}, path)
                if(acc>print_max_acc):
                    print_max_acc = acc

            t_end = time.perf_counter()
            durations.append(t_end - t_start)      
                    
            # Start monitoring CPU and memory usage, model size, number of parametes, time inference and  power consumption
            main_model_path=pathcheck+'/'+args.model+'_'+dataset_name+'_'+str(bit)+'bit'+'main.pth.tar'
            #state = torch.load(main_model_path)
            #dict=state['state_dict']
            #recover_model = lambda: model.load_state_dict(state['state_dict'])

            gc.collect()
            time.sleep(5)  # Add a 5-second delay to stabilize the initial state
            tracemalloc.start()  # Start tracking memory allocations
            snapshot_before = tracemalloc.take_snapshot()#take a snapshot of the current memory state before starting the measurement.

            t0 = time.perf_counter()
            initial_cpu_usage = get_cpu_usage()
            power_usage = estimate_power_usage(initial_cpu_usage)


            fold_main_model_accuracy= eval_acc(model, test_loader)

            fold_main_cpu_usage = get_cpu_usage()
            t1 = time.perf_counter()
            fold_t_main_model=t1-t0

            snapshot_after = tracemalloc.take_snapshot()
            tracemalloc.stop()
            top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

            folde_main_total_memory_diff = sum([stat.size_diff for stat in top_stats])
            fold_main_energy_consumption = power_usage * fold_t_main_model
            fold_main_model_size = os.path.getsize(main_model_path)
            fold_num_parm_main_model=get_num_parameters(model, count_nonzero_only=True)

            gc.collect()
            time.sleep(5) 
            #Update Eva dictionary
            Eva_fold.update({'main model accuracy per fold': fold_main_model_accuracy,
                        'time inference of main model per fold':fold_t_main_model,
                        'number parmameters of main model per fold': fold_num_parm_main_model,
                        'size of main model per fold': fold_main_model_size, 
                        'energy consumption of main model per fold':fold_main_energy_consumption,
                        'total memory usage of main model per fold':folde_main_total_memory_diff,
                        'cpu usage of main model per fold':fold_main_cpu_usage
                       })

            gc.collect()
            time.sleep(5) 


            main_model_accuracy.append(Eva_fold['main model accuracy per fold'])
            t_main_model.append(Eva_fold['time inference of main model per fold'])
            Num_parm_main_model.append(int(Eva_fold['number parmameters of main model per fold']))
            main_model_size.append(int(Eva_fold['size of main model per fold']))
            main_energy_consumption.append(Eva_fold['energy consumption of main model per fold'])
            main_cpu_usage.append(Eva_fold['cpu usage of main model per fold'])
            main_memory_usage.append(Eva_fold['total memory usage of main model per fold'])





        Eva_iter["main model accuracy per iter"]= stat.mean(main_model_accuracy)
        Eva_iter["time inference of main model per iter"]= stat.mean(t_main_model)
        Eva_iter["number parmameters of main model per iter"]=  stat.mean(Num_parm_main_model)
        Eva_iter["size of main model per iter"]= stat.mean(main_model_size)
        Eva_iter["energy consumption of main model per iter"]= stat.mean(main_energy_consumption)
        Eva_iter["cpu usage of main model per iter"]= stat.mean(main_cpu_usage)
        Eva_iter["total memory usage of main model per iter"]= stat.mean(main_memory_usage)


        loss, acc, duration = tensor(val_losses), tensor(accu), tensor(durations)
        loss, acc = loss.view(args.folds, max_epoch), acc.view(args.folds, max_epoch)
        loss, argmin = loss.min(dim=1)
        acc = acc[torch.arange(args.folds, dtype=torch.long), argmin]

        Eva_iter["val losses per iter"]= loss.mean().item()
        Eva_iter["durations per iter"]= duration.mean().item()




        return Eva_iter                                        


## Manual Measurement

In [12]:
# The following are all list of criteria for measurements. 
# We collect all desired datas of each list across iterations. 
# Then, we compute average and std of each list.



#Main model
Main_val_loss=[]
Main_duration=[]
Main_model_accuracy=[]
T_Main_model=[]
Num_parm_Main_model=[]
Main_model_size=[]
Main_Energy_Consumption=[]
Main_Cpu_Usage=[]
Main_Memory_Usage=[]


# Here is the dictionary to record the list of all measurements
Eva_measure={'Main validation loss':Main_val_loss,
             'Main duration':Main_duration,
            'Main model accuracy': Main_model_accuracy,
            'time inference of Main model':T_Main_model,
            'number parmameters of Main model':Num_parm_Main_model,
            'Main model size':Main_model_size,
            'energy consumption of Main model':Main_Energy_Consumption,
            'cpu usage of Main model':Main_Cpu_Usage,
            'memory usage of Main model':Main_Memory_Usage}


In [13]:
args.max_epoch=100
max_epoch = args.max_epoch
iterations=5
args.bit=32
bit=args.bit
folds=10

In [14]:
#### load the Mainized  model

for i in range(iterations):
    print('********************************************')
    print(f'The iteration is :{i+1} ')
   
 

    
    Eva_iter=run(bit, max_epoch)

 
    Main_val_loss.append(Eva_iter["val losses per iter"])
    Main_duration.append(Eva_iter["durations per iter"])
    Main_model_accuracy.append(Eva_iter["main model accuracy per iter"])
    T_Main_model.append(Eva_iter["time inference of main model per iter"])
    Num_parm_Main_model.append(Eva_iter["number parmameters of main model per iter"])
    Main_model_size.append(Eva_iter["size of main model per iter"])
    Main_Energy_Consumption.append(Eva_iter["energy consumption of main model per iter"])
    Main_Cpu_Usage.append( Eva_iter["cpu usage of main model per iter"])
    Main_Memory_Usage.append(Eva_iter["total memory usage of main model per iter"])
 

********************************************
The iteration is :1 
Eval Epoch: 0 |Val_loss:0.585| Train_Loss: 2.228 | Acc: 0.518|Fold: 0
Eval Epoch: 50 |Val_loss:29578.193| Train_Loss: 0.678 | Acc: 0.304|Fold: 0
Eval Epoch: 0 |Val_loss:0.825| Train_Loss: 2.201 | Acc: 0.598|Fold: 1
Eval Epoch: 50 |Val_loss:2.674| Train_Loss: 0.658 | Acc: 0.259|Fold: 1
Eval Epoch: 0 |Val_loss:1.310| Train_Loss: 2.086 | Acc: 0.589|Fold: 2
Eval Epoch: 50 |Val_loss:6.244| Train_Loss: 0.653 | Acc: 0.259|Fold: 2
Eval Epoch: 0 |Val_loss:1.374| Train_Loss: 2.165 | Acc: 0.306|Fold: 3
Eval Epoch: 50 |Val_loss:2.678| Train_Loss: 0.617 | Acc: 0.261|Fold: 3
Eval Epoch: 0 |Val_loss:0.941| Train_Loss: 2.549 | Acc: 0.595|Fold: 4
Eval Epoch: 50 |Val_loss:0.885| Train_Loss: 0.630 | Acc: 0.640|Fold: 4
Eval Epoch: 0 |Val_loss:1.691| Train_Loss: 2.334 | Acc: 0.595|Fold: 5
Eval Epoch: 50 |Val_loss:0.564| Train_Loss: 0.645 | Acc: 0.604|Fold: 5
Eval Epoch: 0 |Val_loss:0.739| Train_Loss: 2.137 | Acc: 0.550|Fold: 6
Eval Epoch: 50

C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Eval Epoch: 0 |Val_loss:1.059| Train_Loss: 2.252 | Acc: 0.598|Fold: 0
Eval Epoch: 50 |Val_loss:46029013361.009| Train_Loss: 0.671 | Acc: 0.321|Fold: 0
Eval Epoch: 0 |Val_loss:1.017| Train_Loss: 1.981 | Acc: 0.277|Fold: 1
Eval Epoch: 50 |Val_loss:90703.796| Train_Loss: 0.640 | Acc: 0.259|Fold: 1
Eval Epoch: 0 |Val_loss:0.871| Train_Loss: 2.382 | Acc: 0.580|Fold: 2
Eval Epoch: 50 |Val_loss:3.504| Train_Loss: 0.641 | Acc: 0.562|Fold: 2
Eval Epoch: 0 |Val_loss:0.763| Train_Loss: 2.804 | Acc: 0.423|Fold: 3
Eval Epoch: 50 |Val_loss:0.853| Train_Loss: 0.686 | Acc: 0.315|Fold: 3
Eval Epoch: 0 |Val_loss:1.519| Train_Loss: 1.941 | Acc: 0.324|Fold: 4
Eval Epoch: 50 |Val_loss:297734.948| Train_Loss: 0.665 | Acc: 0.532|Fold: 4
Eval Epoch: 0 |Val_loss:0.770| Train_Loss: 2.070 | Acc: 0.577|Fold: 5
Eval Epoch: 50 |Val_loss:3736.770| Train_Loss: 0.662 | Acc: 0.405|Fold: 5
Eval Epoch: 0 |Val_loss:0.784| Train_Loss: 2.063 | Acc: 0.477|Fold: 6
Eval Epoch: 50 |Val_loss:0.541| Train_Loss: 0.627 | Acc: 0.595

C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Eval Epoch: 0 |Val_loss:0.573| Train_Loss: 1.888 | Acc: 0.598|Fold: 0
Eval Epoch: 50 |Val_loss:0.557| Train_Loss: 0.660 | Acc: 0.598|Fold: 0
Eval Epoch: 0 |Val_loss:0.854| Train_Loss: 2.092 | Acc: 0.598|Fold: 1
Eval Epoch: 50 |Val_loss:86.126| Train_Loss: 0.657 | Acc: 0.598|Fold: 1
Eval Epoch: 0 |Val_loss:1.020| Train_Loss: 2.286 | Acc: 0.232|Fold: 2
Eval Epoch: 50 |Val_loss:0.536| Train_Loss: 0.660 | Acc: 0.598|Fold: 2
Eval Epoch: 0 |Val_loss:0.970| Train_Loss: 2.125 | Acc: 0.550|Fold: 3
Eval Epoch: 50 |Val_loss:9.586| Train_Loss: 0.637 | Acc: 0.640|Fold: 3
Eval Epoch: 0 |Val_loss:1.463| Train_Loss: 2.373 | Acc: 0.324|Fold: 4
Eval Epoch: 50 |Val_loss:0.547| Train_Loss: 0.658 | Acc: 0.604|Fold: 4
Eval Epoch: 0 |Val_loss:2.171| Train_Loss: 2.188 | Acc: 0.270|Fold: 5
Eval Epoch: 50 |Val_loss:1.537| Train_Loss: 0.616 | Acc: 0.586|Fold: 5
Eval Epoch: 0 |Val_loss:0.912| Train_Loss: 2.058 | Acc: 0.396|Fold: 6
Eval Epoch: 50 |Val_loss:1.861| Train_Loss: 0.658 | Acc: 0.270|Fold: 6
Eval Epoch: 

C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Eval Epoch: 0 |Val_loss:1.057| Train_Loss: 2.180 | Acc: 0.554|Fold: 0
Eval Epoch: 50 |Val_loss:22.675| Train_Loss: 0.635 | Acc: 0.259|Fold: 0
Eval Epoch: 0 |Val_loss:0.656| Train_Loss: 2.023 | Acc: 0.455|Fold: 1
Eval Epoch: 50 |Val_loss:1125.620| Train_Loss: 0.672 | Acc: 0.411|Fold: 1
Eval Epoch: 0 |Val_loss:2.076| Train_Loss: 2.138 | Acc: 0.598|Fold: 2
Eval Epoch: 50 |Val_loss:0.539| Train_Loss: 0.666 | Acc: 0.598|Fold: 2
Eval Epoch: 0 |Val_loss:3.213| Train_Loss: 2.123 | Acc: 0.595|Fold: 3
Eval Epoch: 50 |Val_loss:8018.013| Train_Loss: 0.659 | Acc: 0.270|Fold: 3
Eval Epoch: 0 |Val_loss:0.564| Train_Loss: 2.078 | Acc: 0.649|Fold: 4
Eval Epoch: 50 |Val_loss:308848418446.991| Train_Loss: 0.642 | Acc: 0.270|Fold: 4
Eval Epoch: 0 |Val_loss:0.624| Train_Loss: 2.002 | Acc: 0.577|Fold: 5
Eval Epoch: 50 |Val_loss:36696.269| Train_Loss: 0.620 | Acc: 0.234|Fold: 5
Eval Epoch: 0 |Val_loss:0.709| Train_Loss: 2.172 | Acc: 0.505|Fold: 6
Eval Epoch: 50 |Val_loss:665.226| Train_Loss: 0.644 | Acc: 0.2

C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Eval Epoch: 0 |Val_loss:1.126| Train_Loss: 1.901 | Acc: 0.607|Fold: 0
Eval Epoch: 50 |Val_loss:74.473| Train_Loss: 0.663 | Acc: 0.259|Fold: 0
Eval Epoch: 0 |Val_loss:1.169| Train_Loss: 2.137 | Acc: 0.304|Fold: 1
Eval Epoch: 50 |Val_loss:1807206308.571| Train_Loss: 0.640 | Acc: 0.250|Fold: 1
Eval Epoch: 0 |Val_loss:0.695| Train_Loss: 2.132 | Acc: 0.375|Fold: 2
Eval Epoch: 50 |Val_loss:51623079643.429| Train_Loss: 0.675 | Acc: 0.259|Fold: 2
Eval Epoch: 0 |Val_loss:0.806| Train_Loss: 1.874 | Acc: 0.315|Fold: 3
Eval Epoch: 50 |Val_loss:2.631| Train_Loss: 0.629 | Acc: 0.495|Fold: 3
Eval Epoch: 0 |Val_loss:0.682| Train_Loss: 2.415 | Acc: 0.378|Fold: 4
Eval Epoch: 50 |Val_loss:0.544| Train_Loss: 0.655 | Acc: 0.595|Fold: 4
Eval Epoch: 0 |Val_loss:0.597| Train_Loss: 2.160 | Acc: 0.595|Fold: 5
Eval Epoch: 50 |Val_loss:0.616| Train_Loss: 0.621 | Acc: 0.550|Fold: 5
Eval Epoch: 0 |Val_loss:0.720| Train_Loss: 2.005 | Acc: 0.604|Fold: 6
Eval Epoch: 50 |Val_loss:270.826| Train_Loss: 0.653 | Acc: 0.243

In [17]:
Eva_iter["main model accuracy per iter"]

0.48779761904761904

In [ ]:
# This is a dictionary to save all measurements. Aftre measuring, we can compute mean and std of each item.
from collections import OrderedDict 
Eva_final = OrderedDict()

print(f"All measurement about Main model of type:{ bit} ")   

main_model_val_loss_mean =stat.mean(Main_val_loss)
main_model_val_loss_std = stat.stdev(Main_val_loss)
main_model_val_loss = "{:.3f} ± {:.3f}".format(main_model_val_loss_mean,main_model_val_loss_std)
print("main_model_val_loss - {}".format(main_model_val_loss))


Eva_final.update({'Ave of main loss validation':float(format(main_model_val_loss_mean, '.3f'))})
Eva_final.update({'Std of main loss validation':float(format(main_model_val_loss_std, '.3f'))}) 

###########

main_model_duration_mean =stat.mean(Main_duration)
main_model_duration_std = stat.stdev(Main_duration)
main_model_duration = "{:.3f} ± {:.3f}".format(main_model_duration_mean,main_model_duration_std )
print("main model duration - {}".format(main_model_duration))

Eva_final.update({'Ave of main model duration':float(format(main_model_duration_mean , '.3f'))})
Eva_final.update({'Std of main model duration':float(format(main_model_duration_std, '.3f'))})                                         
                                     
#############
main_model_accuracy_mean =stat.mean(Main_model_accuracy)
main_model_accuracy_std = stat.stdev(Main_model_accuracy)
main_model_accuracy = "{:.3f} ± {:.3f}".format(main_model_accuracy_mean,main_model_accuracy_std )
print("main model accuracy - {}".format(main_model_accuracy))

Eva_final.update({'Ave of main model accuracy':float(format(main_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of main model accuracy':float(format(main_model_accuracy_std, '.3f'))})
                 
######################
t_main_model_mean = stat.mean(T_Main_model)
t_main_model_std =stat.stdev(T_Main_model)
t_main_model = "{:.3f} ± {:.3f}".format(main_model_accuracy_mean,main_model_accuracy_std )
print("time inference of main model- {}".format(t_main_model))
      
Eva_final.update({'Ave of time inference of main model':float(format(t_main_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of main model':float(format(t_main_model_std, '.3f'))})

##############
num_parm_main_model_mean = stat.mean(Num_parm_Main_model)
num_parm_main_model_std = stat.stdev(Num_parm_Main_model)
num_parm_main_model= "{:.3f} ± {:.3f}".format(num_parm_main_model_mean, num_parm_main_model_std)
print("num parm main model: {}".format(num_parm_main_model))


Eva_final.update({'Ave of number parmameters of main model':num_parm_main_model_mean})
Eva_final.update({'Std of number parmameters of main model':num_parm_main_model_std})
##########################
main_model_size_mean =stat.mean( Main_model_size)
main_model_size_std = stat.stdev(Main_model_size)
main_model_size= "{:.3f} ± {:.3f}".format(main_model_size_mean, main_model_size_std)
print("main modelsize: {}".format(main_model_size))


Eva_final.update({'Ave of main model size':main_model_size_mean})
Eva_final.update({'Std of main_model_size':main_model_size_std })

###########################
main_energy_consumption_mean = stat.mean(Main_Energy_Consumption)
main_energy_consumption_std = stat.stdev(Main_Energy_Consumption)
main_energy_consumption= "{:.3f} ± {:.3f}".format(main_energy_consumption_mean, main_energy_consumption_std)
print("main energy consumption: {}".format(main_energy_consumption))


Eva_final.update({'Ave of energy consumption of main model':main_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of main model':main_energy_consumption_std})

###############################
main_cpu_usage_mean = stat.mean(Main_Cpu_Usage)
main_cpu_usage_std = stat.stdev(Main_Cpu_Usage)
main_cpu_usage= "{:.3f} ± {:.3f}".format(main_cpu_usage_mean, main_cpu_usage_std )
print("main cpu usage: {}".format(main_cpu_usage))

Eva_final.update({'Ave of cpu usage of main model':main_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of main model':main_cpu_usage_std})


###############
main_memory_usage_mean = stat.mean(Main_Memory_Usage)
main_memory_usage_std = stat.stdev(Main_Memory_Usage)
main_memory_usage= "{:.3f} ± {:.3f}".format(main_memory_usage_mean, main_memory_usage_std)
print("main memory usage: {}".format(main_memory_usage))

Eva_final.update({'Ave of memory usage of main model':main_memory_usage_mean})
Eva_final.update({'Std of memory usage of main model':main_memory_usage_std})

#################################


# Determing Mainization Method 




In [17]:
# Record the accuracy

mode='FP32'
file_name = pathresult+'/'+args.model+'_'+dataset_name+'_'+str(bit)+'bit'+'_with_Mode_'+mode+'.txt'

with open(file_name, 'w') as f:
    for key, value in vars(args).items():
        f.write('%s:%s\n'%(key, value))

    for key, value in Eva_final.items():
        f.write('%s:%s\n'%(key, value))

    for key, value in Eva_measure.items():
        f.write('%s:%s\n' % (key, ','.join(map(str, value))))    